In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from pathlib import Path

from tensorflow import Tensor
from tqdm.notebook import tqdm
from typing import Callable, List, Tuple, Union, Optional, Dict, Any, Sequence, Iterable, TypeVar
from DataPipe import DataPipeFactory
from util_function import inform_pooling, Get_Gradient

/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/p

In [2]:
# Disable all GPUS
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [3]:
#####
#####6824149711

In [4]:
ds = DataPipeFactory('../DataFolder/Tensorflow_DataRecord/Student_Answer_Record.tfrecord',
                     '../DataFolder/Siri_Related/Siri_Reference_Sample',
                     '../DataFolder/Siri_Related/Siri_Dense_Index', cache='../DataFolder/cache/datapipe/cached')
# dsp = ds.get_batch_data(10)
# it = iter(dsp)
ds.get_raw_data()
A = tf.Variable(-1.)

Load cache from ../DataFolder/cache/datapipe/cached


In [5]:
#ds.pre_save()

In [6]:
for i,d in tqdm(enumerate(ds.get_batch_data(10))):
    batch = 10
    value = tf.RaggedTensor.from_tensor(d['stu_mfcc'], padding=-1*tf.ones(80,))
    start = tf.RaggedTensor.from_tensor(d['valid_stu_start'], padding=-1.)
    duration = tf.RaggedTensor.from_tensor(d['valid_stu_duration'], padding=-1.)
    ratio = 125
    #Get_Gradient(value, start, duration, ratio)
    # print(i)
    # with tf.GradientTape() as t:
    #     v2 = value * A
    #     Final_Tensor = inform_pooling(batch, v2, start, duration, ratio)
    #     J = tf.reduce_mean(Final_Tensor)
    # print('Start Gradient')
    # print(f'\tLoss Value: {J}, \n\tOutput Shape {tf.shape(Final_Tensor)}')
    # G = t.gradient(J,A)
    # print(i,G)
    # print('End Gradient\n\n')
    # del t
    # print(G)
    if i ==30 :break
    pass

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


0it [00:00, ?it/s]

2023-03-06 19:53:48.430694: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


KeyboardInterrupt: 

In [9]:
d.keys()

dict_keys(['stu_mfcc', 'ref_mfcc', 'valid_stu_start', 'valid_stu_duration', 'valid_ref_word', 'valid_ref_start', 'valid_ref_duration', 'counter_ref_mfcc', 'counter_valid_ref_word', 'counter_valid_ref_start', 'counter_valid_ref_duration', 'counter_word_match', 'counter_pool_index'])

In [176]:
class InformPooling(tf.keras.layers.Layer):
    def __init__(self, num_maps, ratios_list, **kwargs):
        super().__init__(**kwargs)
        self.num_maps = num_maps
        self.ratios_list = ratios_list

    @staticmethod
    @tf.function
    def inform_pooling(value, start, duration, ratio, eps=0.001):
        batch = tf.shape(value)[0]
        end = start + duration
        start = tf.math.floor(start * ratio)
        end = tf.math.ceil((end + eps) * ratio)
        period = tf.cast(tf.stack([start, end], axis=-1), tf.int32)
        tf.debugging.assert_less(period[..., 0], period[..., 1])
        ret_b = tf.TensorArray(tf.float32, batch, infer_shape=False)
        ret_count = tf.TensorArray(tf.int64, batch)
        for batch_index in tf.range(batch):
            value_l = value[batch_index]
            val_ind_max = tf.shape(value_l)[0]
            period_l = period[batch_index]
            period_l_p = tf.math.minimum(period_l, val_ind_max - 1)
            ret_count = ret_count.write(batch_index, tf.cast(tf.shape(period_l)[0], tf.int64))
            indexes = tf.ragged.range(period_l_p[..., 0], period_l_p[..., 1])
            value_indices = tf.gather(value_l, indexes)
            pooled = tf.reduce_mean(value_indices, axis=1)
            ret_b = ret_b.write(batch_index, pooled)
        row_length = ret_count.stack()
        ret = ret_b.concat()
        return ret, row_length

    @tf.function
    def call(self, value_list, start, duration):
        # Iterate over both value_list and ratio_list
        pooled_value = [self.inform_pooling(value, start, duration, ratio) for (value, ratio) in
                        zip(value_list, self.ratios_list)]
        ret = tf.concat([val for val, _ in pooled_value], axis=-1)
        # Stupid way to shrink dynamic shape to static shape
        return tf.RaggedTensor.from_row_lengths(ret, pooled_value[0][1])

In [177]:
apl = InformPooling(3,[125 / 2**i for i in range(3)])

In [255]:
def getinfo(d,str_value, str_start, str_duration, str_words):
    batch = 10
    value = tf.RaggedTensor.from_tensor(d[str_value], padding=-1*tf.ones(80,))
    start = tf.RaggedTensor.from_tensor(d[str_start], padding=-1.)
    duration = tf.RaggedTensor.from_tensor(d[str_duration], padding=-1.)
    ratio = 125
    words = tf.RaggedTensor.from_tensor(d[str_words], padding=-1)
    end = start + duration
    period = tf.cast(tf.stack([start, end], axis=-1) * ratio, tf.int64)
    return apl([value]*3, start, duration), words

In [261]:
v1, wo1 = getinfo(d,'stu_mfcc','valid_stu_start','valid_stu_duration', 'valid_ref_word')

In [199]:
d.keys()

dict_keys(['stu_mfcc', 'ref_mfcc', 'valid_stu_start', 'valid_stu_duration', 'valid_ref_word', 'valid_ref_start', 'valid_ref_duration', 'counter_ref_mfcc', 'counter_valid_ref_word', 'counter_valid_ref_start', 'counter_valid_ref_duration', 'counter_word_match', 'counter_pool_index'])

In [262]:
v2, wo2 = getinfo(d, 'ref_mfcc', 'valid_ref_start', 'valid_ref_duration', 'valid_ref_word')

In [201]:
tf.shape(v1)

<DynamicRaggedShape lengths=[10, (25, 17, 16, 17, 19, 17, 24, 23, 44, 13), 240] num_row_partitions=1>

In [202]:
tf.shape(v2)

<DynamicRaggedShape lengths=[10, (25, 17, 16, 17, 19, 17, 24, 23, 44, 13), 240] num_row_partitions=1>

In [276]:
@tf.function
def compute_similarity(value_a,value_b, ref_a, ref_b, margin=0.25, eps=0.001):
    #if ref_a equal ref_b then we consider it should be similar else it should be different,
    #margin prevent it been push to far away
    # compute the norm for ragged tensor
    norm_of_a = tf.sqrt(tf.reduce_sum(tf.square(value_a),axis=-1, keepdims=True))
    norm_of_b = tf.sqrt(tf.reduce_sum(tf.square(value_b),axis=-1, keepdims=True))
    norm_a = value_a / (norm_of_a + eps)
    norm_b = value_b / (norm_of_b + eps)
    # compute cosine similarity for each sample in batch
    # get batch size
    batch_size = tf.shape(norm_a)[0]
    loss_array = tf.TensorArray(tf.float32, batch_size, infer_shape=False)
    for idx in tf.range(batch_size):
        va = norm_a[idx]
        vb = norm_b[idx]
        ra = ref_a[idx]
        rb = ref_b[idx]
        similarity_matrix = tf.matmul(va,vb, transpose_b=True)
        # compute the mask for the positive samples
        mask = tf.cast(tf.equal(ra,rb),tf.float32)
        # compute the mask for the negative samples
        mask_neg = tf.cast(tf.not_equal(ra,rb),tf.float32)
        # compute the number of positive and negative samples
        num_pos = tf.cast(tf.reduce_sum(mask), tf.float32)
        num_neg = tf.cast(tf.reduce_sum(mask_neg), tf.float32)
        # compute the average similarity for the positive samples
        # avoid 0
        num_pos = tf.maximum(num_pos, 1.)
        num_neg = tf.maximum(num_neg, 1.)
        avg_sim_pos = tf.reduce_sum(tf.multiply(similarity_matrix,mask)) / num_pos
        # compute the average similarity for the negative samples
        avg_sim_neg = tf.reduce_sum(tf.multiply(similarity_matrix,mask_neg)) / num_neg
        # compute the max similarity for the positive samples
        max_sim_pos = tf.reduce_max(tf.multiply(similarity_matrix,mask))
        # compute the min similarity for the negative samples
        min_sim_neg = tf.reduce_min(tf.multiply(similarity_matrix,mask_neg))
        # compute the average loss with margin
        loss_avg = tf.maximum(0., margin + avg_sim_pos - avg_sim_neg)
        # compute min_max loss with margin
        loss_min_max = tf.maximum(0., margin + max_sim_pos - min_sim_neg)
        # total loss
        loss = loss_avg + loss_min_max
        loss_array = loss_array.write(idx,loss)
    total_loss = tf.reduce_sum(loss_array.stack())
    return total_loss

In [277]:
compute_similarity(v1,v2,wo1,wo2)

<tf.Tensor: shape=(), dtype=float32, numpy=78.2796>

In [254]:
d['valid_ref_word']

<tf.Tensor: shape=(10, 44), dtype=int64, numpy=
array([[ 118, 1589,  647, 1604, 1494, 1459,  670,   71,  647,  595,  722,
          44,  730,  713,  976, 1434,  820, 1138, 1497, 1614, 1467, 1420,
         713, 1459, 1434,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,
          -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
       [1459, 1490,  647, 1585,  820,  838, 1494, 1274,  647, 1459, 1459,
        1459,   44,   36,   71, 1467, 1068,   -1,   -1,   -1,   -1,   -1,
          -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,
          -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
       [ 813,  713, 1459,  614, 1251,  450,  813, 1610,    0,  489,  844,
         987, 1557, 1496, 1251,  413,   -1,   -1,   -1,   -1,   -1,   -1,
          -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,
          -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
       [ 864,   44,  129,  550, 1467,    0, 1275, 1467, 1605,

In [218]:
c = tf.matmul(v1,v2, transpose_b=True)

In [238]:
c.nested_row_lengths()

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([25, 17, 16, 17, 19, 17, 24, 23, 44, 13])>,
 <tf.Tensor: shape=(215,), dtype=int64, numpy=
 array([25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
        25, 25, 25, 25, 25, 25, 25, 25, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 24, 24, 24, 24, 24, 24, 24, 24,
        24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 23,
        23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
        23, 23, 23, 23, 23, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
        44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
        44, 44, 44, 44

In [239]:
rt = tf.RaggedTensor.from_row_splits(
    values=tf.RaggedTensor.from_row_splits(
        values=[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        row_splits=[0, 3, 3, 5, 9, 10]),
    row_splits=[0, 1, 1, 5])

In [240]:
rt.nested_row_lengths()

(<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 0, 4])>,
 <tf.Tensor: shape=(5,), dtype=int64, numpy=array([3, 0, 2, 4, 1])>)

In [10]:
ap = inform_pooling(value, start, duration, ratio)

In [15]:
tf.shape(ap)

<DynamicRaggedShape lengths=[10, (62, 19, 22, 37, 133, 27, 77, 27, 42, 26), 80] num_row_partitions=1>

In [17]:
tf.shape(tf.concat([ap,ap],axis=-1))

<DynamicRaggedShape lengths=[10, (62, 19, 22, 37, 133, 27, 77, 27, 42, 26), 160] num_row_partitions=1>

In [46]:
down_sample_layer = tf.keras.layers.Conv1D(120,kernel_size=3,strides=2,padding="same")

In [47]:
up_sample_layer = tf.keras.layers.Conv1DTranspose(120,kernel_size=4,strides=2,padding="same")

In [53]:
i = d['stu_mfcc']

In [54]:
s = down_sample_layer(i)

In [55]:
u = up_sample_layer(s)

In [56]:
u

<tf.Tensor: shape=(10, 7956, 120), dtype=float32, numpy=
array([[[-3.3503982e-01, -7.4509400e-01,  3.9917171e-01, ...,
          8.3293056e-01,  6.1320883e-01, -1.2726359e+00],
        [ 3.7982881e-02, -4.9109459e-01, -1.4010111e+00, ...,
          1.9802206e+00, -1.1918473e+00,  5.0352591e-01],
        [ 4.2172909e-02, -5.7656789e-01,  1.5796090e+00, ...,
          2.6484108e+00,  1.3580958e+00, -4.5213699e-02],
        ...,
        [ 5.1405555e-01,  6.9830954e-01, -5.1836073e-01, ...,
          8.3936220e-01, -7.3919445e-01,  1.0372110e-01],
        [-1.5835464e-02, -6.7796946e-02,  3.6592585e-01, ...,
          1.4088244e+00,  3.3611232e-01, -1.9279461e-01],
        [-2.4554195e-01,  5.7582664e-01,  1.4301073e-03, ...,
          1.3548607e-01, -3.3273369e-01, -2.3910619e-01]],

       [[-2.1094964e-01, -4.6145675e-01,  2.1487795e-01, ...,
          5.8664572e-01,  1.8823710e-01, -6.9060826e-01],
        [ 6.1649752e-01,  1.1193681e-01, -1.1283131e+00, ...,
          1.0929437e+00, -

In [57]:
i

<tf.Tensor: shape=(10, 7955, 80), dtype=float32, numpy=
array([[[-6.6438591e-01, -3.5176598e-02, -1.8905866e+00, ...,
         -2.6070287e+00, -2.8596351e+00, -3.3041153e+00],
        [-6.0051984e-01, -1.7968614e-02, -1.8218514e+00, ...,
         -2.4144011e+00, -2.6640015e+00, -3.0863380e+00],
        [-6.4661127e-01, -8.8636029e-05, -1.7614514e+00, ...,
         -2.3220947e+00, -2.6222863e+00, -3.0480011e+00],
        ...,
        [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
         -1.0000000e+00, -1.0000000e+00, -1.0000000e+00],
        [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
         -1.0000000e+00, -1.0000000e+00, -1.0000000e+00],
        [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
         -1.0000000e+00, -1.0000000e+00, -1.0000000e+00]],

       [[-2.0615819e+00, -1.0019447e+00, -8.3451664e-01, ...,
         -2.4256451e+00, -2.7573261e+00, -3.2293723e+00],
        [-1.8697287e+00, -1.2216491e+00, -9.3900388e-01, ...,
         -2.4965630e+00, -2

In [62]:
u[:,:tf.shape(i)[2],:]

<tf.Tensor: shape=(10, 80, 120), dtype=float32, numpy=
array([[[-3.35039824e-01, -7.45094001e-01,  3.99171710e-01, ...,
          8.32930565e-01,  6.13208830e-01, -1.27263594e+00],
        [ 3.79828811e-02, -4.91094589e-01, -1.40101111e+00, ...,
          1.98022056e+00, -1.19184732e+00,  5.03525913e-01],
        [ 4.21729088e-02, -5.76567888e-01,  1.57960904e+00, ...,
          2.64841080e+00,  1.35809577e+00, -4.52136993e-02],
        ...,
        [-4.48540539e-01, -3.78239393e-01, -1.11343288e+00, ...,
          2.14028788e+00, -8.66763890e-01, -2.02339888e-01],
        [ 1.61549211e-01, -1.21813416e-01,  2.64924955e+00, ...,
          4.46287537e+00,  9.71228004e-01, -6.14421964e-01],
        [-3.51147652e-01, -1.35940552e-01, -8.80091310e-01, ...,
          2.03196859e+00, -9.50756550e-01, -2.87807733e-01]],

       [[-2.10949644e-01, -4.61456746e-01,  2.14877948e-01, ...,
          5.86645722e-01,  1.88237101e-01, -6.90608263e-01],
        [ 6.16497517e-01,  1.11936808e-01, -1.12

In [61]:
tf.ensure_shape(i)

TypeError: Missing required positional argument

In [82]:
@tf.function
def mt(i):
    shape = tf.shape(i)[1]
    s = down_sample_layer(i)
    u = up_sample_layer(s)
    cut = u[:,:shape]
    return tf.concat([i,cut],axis=-1)


In [83]:
mt(i)

<tf.Tensor: shape=(10, 7955, 200), dtype=float32, numpy=
array([[[-6.6438591e-01, -3.5176598e-02, -1.8905866e+00, ...,
          8.3293056e-01,  6.1320883e-01, -1.2726359e+00],
        [-6.0051984e-01, -1.7968614e-02, -1.8218514e+00, ...,
          1.9802206e+00, -1.1918473e+00,  5.0352591e-01],
        [-6.4661127e-01, -8.8636029e-05, -1.7614514e+00, ...,
          2.6484108e+00,  1.3580958e+00, -4.5213699e-02],
        ...,
        [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
          1.5338720e+00,  3.3500841e-01, -3.5402846e-01],
        [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
          8.3936220e-01, -7.3919445e-01,  1.0372110e-01],
        [-1.0000000e+00, -1.0000000e+00, -1.0000000e+00, ...,
          1.4088244e+00,  3.3611232e-01, -1.9279461e-01]],

       [[-2.0615819e+00, -1.0019447e+00, -8.3451664e-01, ...,
          5.8664572e-01,  1.8823710e-01, -6.9060826e-01],
        [-1.8697287e+00, -1.2216491e+00, -9.3900388e-01, ...,
          1.0929437e+00, -

In [79]:
shape = tf.shape(i)[:-1]
cut = tf.slice(u, tf.zeros_like(shape), shape)

InvalidArgumentError: {{function_node __wrapped__Slice_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expected begin and size arguments to be 1-D tensors of size 3, but got shapes [2] and [2] instead. [Op:Slice]

In [81]:
shape

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([  10, 7955], dtype=int32)>

In [180]:
rt = tf.RaggedTensor.from_uniform_row_length(
    values=tf.RaggedTensor.from_row_splits(
        values=[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        row_splits=[0, 3, 5, 9, 10]),
    uniform_row_length=2)

In [186]:
rt.nested_row_splits

(<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 2, 4])>,
 <tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 0,  3,  5,  9, 10])>)

In [191]:
rt.row_splits

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 2, 4])>

In [193]:
rt.values

<tf.RaggedTensor [[10, 11, 12], [13, 14], [15, 16, 17, 18], [19]]>

In [194]:
rt

<tf.RaggedTensor [[[10, 11, 12], [13, 14]],
 [[15, 16, 17, 18], [19]]]>